# Logbook/Diary

In this notebook I am going to write a 'diary' of my thoughts and general directions during my project. I expect it to be quite informal, and it should be useful when writing the final report. Also it's fairly good practice within research.

Saving for future reference:
- https://github.com/ltd3141/On-Low-Thermal-Conductivity-Materials
- https://github.com/hannahdauv/Masters_Project

## 1 October 2025
After having met with my supervisor I thought it would be a good idea to start getting myself familiarised with LAMMPS. I began by reading the documentation on LAMMPS (https://docs.lammps.org/) to understand how to install and run the program.

From the LAMMPS Windows Installer website (https://rpm.lammps.org/windows/), I downloaded the 64-bit stable Python version, which I believe contains the LAMMPS package with added Python versatility. Originally I had downloaded the non-MSMPI version to not overcomplicate things. Once I messed around a bit with LAMMPS, I went back and downloaded the MSMPI version.

I decided to try to learn how to run LAMMPS with `in.crack.lmp` which is found in the examples folder. This simulates the propagation of a crack in a two-dimensional system. I'm quite happy I chose to investigate this, because I learnt how to visualise my results in a movie!

It did take me an embarassingly long amount of time to realise that I needed to use the Windows Command Prompt to run LAMMPS simulations. When I want to simulate something with LAMMPS, I do these two things in order:

1. Change the directory command prompt uses to the directory where our LAMMPS files are using the prompt
`cd your\path\to\LAMMPS\examples`, here `cd` stands for "change directory"
2. Run the simulation using `lmp -in [your_file_name.lmp]`

I found the README files quite helpful here because they explained the cmd prompt outputs pretty well. I really wanted to do more, but I had to focus on tomorrow's data analysis lecture.

## 2 October 2025
I went back and installed the MSMPI version of LAMMPS. When I read through both dissertations, I thought that overall there wasn't that many simulations which actually took place (not to discredit anyone's work, obviously). This suggested to me that LAMMPS simulations could take a really long time. MSMPI version should speed up LAMMPS simulations, as it allows me to use parallel processing (this is discussed in Ch 6.6 of the documentation, but I remember seeing it somewhere else aswell). My computer has a CPU with 8 cores and 16 threads, so I think in theory I should be able to achieve a 16-fold efficiency boost.

Unfortunately downloading MSMPI didn't come without its struggles as I also needed to install MSMPI's dll packages (https://www.microsoft.com/en-us/download/details.aspx?id=105289) in order to utilise the `mpiexec` and `mpirun` commands. I found out in a pretty annoying way that I needed to install these in the C drive rather than the E drive. I think it's because cmd prompt can't access the commands if they're saved in different drives.

Having figured out how to get `mpiexec` running I decided to test finding the thermal conductivity of argon, much more in line with the experiment. Daniel's file depended on an external .txt file which I couldn't find; I used Hannah's input file instead, which worked.

I used the basic `lmp -in file.lmp` which uses just one processor and it managed to get the simulation done in a few minutes and it got a reasonably close thermal conductivity to the true one of argon. I then tried with 8 processors using the command `msiexec -np 8 lmp -in file.lmp` and this got similar results albeit it was noticeably much quicker. I then tried 16 processors but my CPU began making a screeching noise; safe to say that command won't be run again.

Interestingly I seemed to find a different thermal conductivity for the three cases, but they were all around the k=0.25W/Km^2 value of argon's thermal conductivity. This suggests that if I want to adequately assess LAMMPS viability in calculating thermal conductivity, I'd need to run multiple simulations and average the result. I also still need to learn some more about the LAMMPS documentation, e.g. how to construct these input files.

A direction I want to take now is to continue understanding how LAMMPS works, but also I want to try to find literature regarding many-body localisation's link to thermal conductivity.

## 6 October 2025
I read a bit more about LAMMPS documentation, regarding commands, trying to understand how I can construct LAMMPS files. I thought the best way to do this would be to try to reproduce a thermal conductivity simulation of argon by myself.
Argon takes shape as a face-centered cubic (FCC) for most temperatures. For ultralow temperatures it crystallises in a HCP arrangement instead, but for simplicity I'll use the FCC arrangement.

The following configuration

```
# Set up lattice
lattice	custom	5.256	origin 0.0 0.0 0.0 &
                              orient x 1 0 0 orient y 0 1 0 orient z 0 0 1 &
                              basis 0.0 0.0 0.0 basis 0.5 0.5 0.0 basis 0.5 0.0 0.5 basis 0.0 0.5 0.5
```

is the same as

```
# Set up lattice
lattice fcc 5.256
```

The lattice constant of argon is 5.256 angstrom, taken from https://www.webelements.com/argon/crystal_structure.html, which references D. G. Henshaw, Physical Review, 1958, 111, 1470. Also found to be 5.256 angstroms in https://periodictable.com/Elements/018/data.html. Both sources have argon's relative atomic mass at 39.948g/mol.

In LAMMPS we also need the Lennard-Jones parameters of Argon to be able to describe the pairwise potential. Two sources I found have $\sigma = 3.400$ (DOI:10.3329/jce.v29i1.33820) and $\sigma = 3.405$ angstroms (must be this one: https://doi.org/10.1093/oso/9780198803195.001.0001). Correspondingly these two sources found $\epsilon = 0.0104$eV and $\frac{\epsilon}{k_B} = 119.82$K.

In my LAMMPS simulation I decided to use real units, as this seemed the most simple for the current stage. Real units take the epsilon Lennard-Jones parameter in kcal/mol so this requires quite a bit of conversion. (kcal = 4.184J)

0.0104 x (1.6x10^-19) / 4.184 obtained in kcal, times by 6.022x10^23 to get it in kcal/mol

119.82K x (1.38x10^-23) J/K / 4.184 obtained in kcal, times by 6.022x10^23 to get it in kcal/mol

And I can easily faciltate the calculation of these numbers in the configuration file using stored variables.

### Looking at KAPPA examples in LAMMPS
I only just found out that LAMMPS actually goes through examples to calculate thermal conductivity, in a folder called KAPPA. Here they list five different methods of calculating it for a Lennard-Jones fluid:
#### `in.langevin`
Here we thermostat 2 regions at different temperatures using fix langevin. I believe by doing this, we can obtain a temperature gradient across the simulation box, and I remember that thermal conductivity is directly proportional to the temperature gradient from Section 4 of SSP. Uses `fix.langevin`. To enforce a constant temperature, we want to scale particle velocities in the region whilst preserving overall momentum. We do that by subtracting energy on one end and adding the same amount of energy to the other end?
#### `in.heat`
I also remember reading about this in Daniel's report. Instead of setting up a temperature gradient we look at the problem from an energy perspective, adding and subtracting equal amounts of energy to two fixed regions via `fix heat`. According to Daniel's report, `fix heat` enforces a constant energy density in the thermostatted regions by adding and subtracting a given amount of energy each timestep. As before, the added and subtracted energies must be equal.
#### `in.ehex`
A newer, improved version of the `fix heat` command fixing the energy drift problem in the previous.
#### `in.mp`
Uses `fix thermal/conductivity` to calculate $\kappa$. Specifically it uses the Muller-Plathe method... I'll look into that method later
#### `in.heatflux`
If I remember correctly, this was the only equilibrium technique, as such the system should be equilibrated before using this. But I also do remember it being referred to as the most superior one, so we'll take a look. Uses `compute heat/flux` and the Green-Kubo method, which has been described in both reports to great detail already.

Unfortunately my replication of calculating the thermal conductivity of argon didn't work. I'm not quite sure what happened yet, and I also am worrying about my literature review.

## 7, 8 October 2025
I decided to spend my next two days starting my literature review. I know it's due in a month but I thought that I could close any gaps in my knowledge by starting now. Also I could get feedback on it as I develop it. I'm thinking about writing a very long version of it, then slowly condensing that long version down into the important bits needed for the literature review coursework. The long version could stay for the final report.

I think I need a bit more experience with LAMMPS still. I don't feel 100% confident, and I don't think I know most of the commands yet. I think the more I know, perhaps the more ideas I could have to minimise thermal conductivity, this could be quite useful.